# NUSTAR SCIENTIFIC PRODUCTS EXTRACTION

This notebook contains the set of cells which extract the NUSTAR scientific products from the raw data and prepares the data for further analysis (not phase-resolved  spectroscopy).

-------

First, import necessary classes and functions from `nustar_scripts` folder in the root

In [4]:
import sys
sys.path.append('../../')
from nustar_scripts.nu_class import np, plt, os, glob, NustarObservation
from nustar_scripts.sj0243_init import Nu_path, ObsList_bright, ObsList, periods_val


In [5]:
print(ObsList)

['90201041002']


------

### Choose the  ObsID you want to work with by setting `ObsID=ObsList[...]`, or by  setting it by  your value, e.g.  `ObsID = '90201041002'`

In [6]:
ObsID =  '90201041002'#ObsList[0]
nu_obs = NustarObservation(ObsID, nu_path=Nu_path)

###
Observation 90201041002 loaded successfully


1. Run this cell which creates nupipeline  and  source/background  regions extraction scripts.
2. After the cell finishes, follow instructions one cell below.

In [7]:
#######   nupipeline and regions   #######
nu_obs = NustarObservation(ObsID, nu_path=Nu_path)
nu_obs.nupipeline(ObsList_bright=ObsList_bright)
nu_obs.make_regions()

###
Observation 90201041002 loaded successfully


Exception: nupipeline has already been launched!

1. Run `nupipeline.sh` script in the `out{ObsID}/` directory and wait  until  the nupipeline exits. Check for the errors in nupipeline (it may take some time).
2. Run `ds9_set.sh` sript to set the region files **for both detectors**. Detector `FPMA` would be on the left side of ds9 view, `FPMB` - on the right. Save regions as `srcA.reg`, `srcB.reg`, `bkgA.reg`, and `bkgB.reg` in `out{ObsID}/` folder.
3. Run `ds9_check.sh` script to check the region files. 

-------

### basic scientific products extraction
extracts spectra and light curves from both modules. 
1. Run the cell and then follow instructions one cell below. 
2. Set parameters as you with (see `nuproducts` method docstring). You may change the energy range of interest, light curve binsize, and a number of other parameters.

In [9]:
prodpath_ave = 'spe_and_lc' #folder name for basic nustar products
for mode in ['A', 'B']:      
    nu_obs.nuproducts(outdir=prodpath_ave, mode=mode, stemout=prodpath_ave ,binsize='0.01') #set light curve bin size in seconds and other parameters of interest



Creating command command: 
    nuproducts     indir=/Users/sdbykov/work/xray_pulsars/nustar_sj0243/nustar_products/out90201041002     instrument=FPMA     steminputs=nu90201041002     stemout=spe_and_lcA     outdir=spe_and_lc     srcregionfile=/Users/sdbykov/work/xray_pulsars/nustar_sj0243/nustar_products/out90201041002/srcA.reg     bkgextract=yes     bkgregionfile=/Users/sdbykov/work/xray_pulsars/nustar_sj0243/nustar_products/out90201041002/bkgA.reg     binsize=0.01     lcfile=DEFAULT     phafile=DEFAULT     bkglcfile=DEFAULT     bkgphafile=DEFAULT     imagefile=DEFAULT     usrgtifile=NONE     pilow=60 pihigh=1935     lcenergy=10     usrgtibarycorr=no    runmkarf=yes    runmkrmf=yes
Writing to file:  spe_and_lcA
Creating command command: 
    nuproducts     indir=/Users/sdbykov/work/xray_pulsars/nustar_sj0243/nustar_products/out90201041002     instrument=FPMB     steminputs=nu90201041002     stemout=spe_and_lcB     outdir=spe_and_lc     srcregionfile=/Users/sdbykov/work/xray_pulsars/nu

1. Go to `out{ObsID}/products` directory, run `spe_and_lcA.sh` and `spe_and_lcB.sh` scripts.
2. Go to `out{ObsID}/products/spe_and_lc`, some L1 products should be created and almost prepared for analysis (need to excecute the next cell)

-----

1. Run the code to make scripts for the extraction of barycentred light curves and grouping energy spectra. Set parameters as you with (see functions annotations).

In [14]:

lclist = [f'{prodpath_ave}{mode}_sr.lc' for mode in ['A', 'B']]
spelist = [f'{prodpath_ave}{mode}_sr.pha' for mode in ['A', 'B']]

nu_obs.grppha(infiles=spelist,
              prodpath='spe_and_lc', group_min=25) #set minimum grouping of spectra

nu_obs.barycorr(infiles=lclist, prodpath=prodpath_ave, barytime='no')


Creating command command: grppha infile="spe_and_lcA_sr.pha" outfile="spe_and_lcA_sr.pi"  comm="group min 25 & exit" clobber=yes
Writing to file:  grppha
Creating command command: grppha infile="spe_and_lcB_sr.pha" outfile="spe_and_lcB_sr.pi"  comm="group min 25 & exit" clobber=yes
Writing to file:  grppha
Creating command command:       barycorr         infile=spe_and_lcA_sr.lc         outfile=spe_and_lcA_sr.lc_bary        orbitfiles=/Users/sdbykov/work/xray_pulsars/nustar_sj0243/nustar_data/90201041002/auxil/nu90201041002_orb.fits.gz        barytime=no 
Writing to file:  barycorr
Creating command command:       barycorr         infile=spe_and_lcB_sr.lc         outfile=spe_and_lcB_sr.lc_bary        orbitfiles=/Users/sdbykov/work/xray_pulsars/nustar_sj0243/nustar_data/90201041002/auxil/nu90201041002_orb.fits.gz        barytime=no 
Writing to file:  barycorr


1. Run  `grppha.sh` to regroup energy spectra.
2. Run `barycorr.sh` to  make barycentred light curves of the source. 


**Note** if your source is bright, the light curves for background should probably be also barycentred and subtracted from the source light curves.

### sum light curves from modules A and B, correct times of arrival with orbital parameters if necessary

In [ ]:
#sum light curves and make orbital correction
nu_obs.lcmath(infiles=lclist,
              outfile=prodpath_ave+'AB_sr.lc', prodpath=prodpath_ave, cmd_name='lcmath_orig',rewrite=True)

nu_obs.lcmath(infiles=[x+'_bary' for x in lclist],
              outfile=prodpath_ave+'AB_sr.lc_bary', prodpath=prodpath_ave, cmd_name='lcmath_bary', rewrite=True)


In [ ]:

#first you launch lcmath and then apply orbital correction
nu_obs.orb_correction_lc(
    filename=prodpath_ave+'AB_sr.lc_bary', prodpath=prodpath_ave)


### search for orbital period and make pulse profile of the light curve 
 After you have found the period (as the maximum of chi2 value, or fitting efsearch results with gaussian, write the prriod for your ObsID into a dictionaty periods_val in init file of your pulsar)

In [ ]:
####### orbital period search and light curve folding #######

nu_obs.make_efsearch(prodpath_ave+'AB_sr.lc_bary_orb_corr',
                     prodpath=prodpath_ave, p0='9.823', dres = '0.001', nper='64', nphase=16, cmd_name='efsearch_AB', rewrite=True)


In [ ]:
period = periods_val[ObsID]
nu_obs.make_efold(filename = prodpath_ave+'AB_sr.lc_bary_orb_corr', prodpath = prodpath_ave, period = str(period), cmd_name='efold_AB', nphase='128')